# Extracting Avg Price for different types of homes from Trulia.com and Merging city demographics data

In [ ]:

# Importing all the required packages 

import time
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

#selenium packages
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

def stripFun(x):
    x=x.strip().split("(")[0]
    return x.strip()

#Settings required to be in incognito mode
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
baseUrl="https://www.trulia.com"

#Instantiating the chromeDriver . Need explicit installation of selenium and Chrome WebDriver
driver = webdriver.Chrome()
driver.get(baseUrl)
time.sleep(5)

#Defining the cities List.Will be using these cities to query the website and get the details
citiesList=["San Jose, CA","Milpitas, CA","Fremont, CA","Hayward, CA","Castro Valley, CA",
            "Berkeley, CA","Oakland, CA","Dublin, CA","San Ramon, CA",
            "Danville, CA","Union City, CA","Palo Alto, CA","Santa Clara, CA",
            "Cupertino, CA","San Mateo, CA","Burlingame, CA","Pleasanton, CA","Newark, CA",
            "Concord, CA","Walnut Creek, CA","Santa Cruz, CA","Half Moon Bay, CA"]

#Dictionary to hold values for each City
priceDict={}
#Parent loop tp iterate over cities
try:
    for j in  range(0,len(citiesList)-21):
        city=str(citiesList[j].split(",")[0]).strip().replace(" ","_")
        state=str(citiesList[j].split(",")[1]).strip()
        element=WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH, "//input[@data-testid='location-search-input']")))
        try:
            element.click()
            element.send_keys(Keys.CONTROL + "a");
            time.sleep(3)
            element.send_keys(Keys.DELETE);
            #Sending the city value here. This action will populate the search bar of the website with the given value
            element.send_keys(citiesList[j])
            
            #Trigerring the click event on the Search button
            driver.find_element_by_xpath("//div[@data-testid='location-search-button']").click()
            time.sleep(10)
        except:
            print("Exception in getting Search Box")
            driver.get(baseUrl+"/"+state+"/"+city+"/");
            times.sleep(10)
        
        #Following code triggers the scroll event so that dynamic HTML could be updated
        element = driver.find_element_by_xpath("//*[@id='resultsColumn']/nav")
        driver.execute_script("return arguments[0].scrollIntoView(true);", element)
        time.sleep(10)
        page_source = driver.page_source
        soup =  BeautifulSoup(page_source,'lxml')
       
       #Check if dynamic HTML get created;in case it goes to exception, reload the page and repeat the same
        try:
            reviewEle2=WebDriverWait(driver,15).until(EC.presence_of_element_located((By.XPATH, "//*[@id='resultsColumn']/div[3]/div/div/div[1]/div/div[2]")))
          
        except:
            print("Inside exception1")
            driver.get(baseUrl+"/"+state+"/"+city+"/");
            time.sleep(10)
            element = driver.find_element_by_xpath("//*[@id='resultsColumn']/nav")
            driver.execute_script("return arguments[0].scrollIntoView(true);", element)
            time.sleep(10)
            
        #Getting Affordibiility range values in each city
        try:
            div1 = soup.find('h3', text = re.compile('Affordability of Living')).find_parent('div').find_next_sibling('div')
            avgHomeValue=div1.contents[0].get_text()
            div2=div1.find_next_sibling('div')
            avgValueperUnit=div2.contents[0].get_text()
        except:
            print("Inside exception2")
            priceInfo['AvgHomePrice']=None
            priceInfo['AvgPricePerUnit']=None
            
        
        #Get avg price of 1 bedroom,2 bedroom ,3 bedroom ,4 bedroom houses over the last month for each city
        tables=pd.read_html(page_source)
        
        #Transpose the table and get the details
        priceInfo=tables[0].transpose()
        priceInfo.reset_index(drop=True,inplace=True)
        
        #Clean the column Names
        priceInfo.iloc[0]=priceInfo.iloc[0].apply(stripFun)
        priceInfo.columns=priceInfo.iloc[0]
        
        #Remove the second row
        priceInfo=priceInfo[1:]
        colList=["2 bedrooms","3 bedrooms","4 bedrooms","1 bedroom"]
        for i in colList:
            if i not in priceInfo.columns.to_list():
                priceInfo[i]=None
            else:
                pass
        priceInfo['AvgHomePrice']=avgHomeValue
        priceInfo['AvgPricePerUnit']=avgValueperUnit
        priceInfo['City']=city
        priceDict[city]=priceInfo
    #CLose the driver at the end of loop
    driver.quit()
       
    print(priceDict)
except:
    print("Exception",Exception.with_traceback())
    driver.quit()
    
    
#Merging all the city details into single Dataframe and writing it t the csv file
        
cityHousePricedf=pd.concat(list(priceDict.values()),ignore_index=True)


#### Cleaning the dataset and writing to the csv file

In [ ]:
def cleanCityDataset(cityHousePricedf):
    colList=list(cityHousePricedf.columns)
    cityHousePricedf['City']=cityHousePricedf['City'].apply(lambda x:str(x).replace("_",""))
    #Clean Avg Home price and Avg PricePerUnit
    cityHousePricedf['AvgHomePrice']=cityHousePricedf['AvgHomePrice'].apply(lambda x:re.sub(r'[^\d.]+', '', str(x)))
    #Clean BPovertyLine
    cityHousePricedf['AvgPricePerUnit']=cityHousePricedf['AvgPricePerUnit'].apply(lambda x:re.sub(r'[^\d.]+', '', str(x)))
    for i in colList:
        cityHousePricedf[i]=cityHousePricedf[i].apply(lambda x:str(x).strip("$").replace(',',''))
    return cityHousePricedf

In [ ]:
cityHousePricedf=cleanCityDataset(cityHousePricedf)
cityHousePricedf.to_csv("AvgPricePerCity.csv",index=False)
cityHousePricedf.head()

###  Merge CrimeIndex information to this file


In [ ]:
#Reading the data collected from above scraping process
cityHousePricedf=pd.read_csv("AvgPricePerCity.csv")

#CrimeDetailsperCity.csv contains Crime Index information for each city
cityCrimeIndexdf=pd.read_csv("CrimeDetailsperCity.csv")
uniqueCities=cityCrimeIndexdf['city'].unique().tolist()

In [ ]:
#Add CrimeIndes column to the Avg Home Price per City Dataset
for i in range(0,len(uniqueCities)):
    city=str(uniqueCities[i])
    
    if(cityCrimeIndexdf[cityCrimeIndexdf['city']==str(city)].size > 0):
        cityHousePricedf.loc[cityHousePricedf['City']==city,'CrimeIndex']=cityCrimeIndexdf[cityCrimeIndexdf['city']==city]['crimeIndex'].item()



In [ ]:
cityHousePricedf.to_csv("AvgPrice_CrimeDetails.csv",index=False)
cityHousePricedf.head()